In [ ]:
#This project is about music composition using AI
#We mainly focused on the music of Piano
#We used LSTM, a Recurrent Neural Network(RNN) approach
#Platform : Google Colab
#Libraries : Tensorflow,Music21,Keras,NumPy,Sklearn,tqdm

In [ ]:
from google.colab import files
#upload zip file of All_Midi_Files given
path_to_file = list(files.upload().keys())[0]

Saving All_Midi_Files.zip to All_Midi_Files (1).zip


In [ ]:
!apt-get install poppler-utils
!unzip /content/All_Midi_Files.zip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 0s (2,081 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123588 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...
Archive:  /content/All_Midi_Files.zip
 extracting: All Midi Files/albeniz/alb_esp1.mid  
 extracting: All Midi Files/albeniz/alb_esp2.mid  
 extracting: All Midi Files/albeniz/alb_esp3.mi

In [ ]:
from music21 import *
import glob
import numpy as np
import pandas as pd
from tqdm import tqdm
from tensorflow.keras.layers import LSTM,GRU,Dense,Input,Dropout,Flatten
from tensorflow.keras.models import Sequential,Model,load_model
from sklearn.model_selection import train_test_split
import random

In [ ]:
#Reading and parsing function
def read_file(file):
  notes=[]
  notes_to_parse=None
  midi=converter.parse(file)
  instrmt=instrument.partitionByInstrument(midi)

  #Fetching Piano Data
  for part in instrmt.parts:
    if 'Piano' in str(part):
      notes_to_parse=part.recurse()

#checking element type is Note or chord
 # if element is chord, we split it into notes
      for element in notes_to_parse:
        if type(element)==note.Note:
          notes.append(str(element.pitch))
        elif type(element)==chord.Chord:
          notes.append('.'.join(str(n) for n in element.normalOrder))
  return notes

In [ ]:
file_path=["haydn"]
all_files=glob.glob('All Midi Files/'+file_path[0]+'/*.mid', recursive=True)

#reading each midi file
notes_list= [read_file(i) for i in tqdm(all_files, position=0,leave=True)]

# Pad the sequences to have the same length
max_length = max(len(lst) for lst in notes_list)
padded_notes = [lst + [''] * (max_length - len(lst)) for lst in notes_list]

# Now create the NumPy array
notes_array = np.array(padded_notes)

100%|██████████| 21/21 [00:13<00:00,  1.61it/s]


In [ ]:
#making array of unique notes
# notess = sum(notes_array,[])
notess = [item for sublist in notes_array for item in sublist] # Flatten the list of lists
unique_notes = list(set(notess))
print("Unique Notes:",len(unique_notes))

#notes with their frequency
freq = dict(map(lambda x: (x,notess.count(x)),unique_notes))

#getting the threshold frequency
print("\nFrequency notes")
for i in range(30,100,20):
  print(i,":",len(list(filter(lambda x:x[1]>=i,freq.items()))))
#freq_notes = []

#filtering notes >50
freq_notes = dict(filter(lambda x:x[1]>=50, freq.items()))
new_notes = [[i for i in j if i in freq_notes] for j in notes_array]

#dictionary having key as note index and value as note
ind2note = dict(enumerate(freq_notes))

#reverse of above dictionary
note2ind = dict(map(reversed,ind2note.items()))



Unique Notes: 180

Frequency notes
30 : 93
50 : 76
70 : 67
90 : 58


In [ ]:
timesteps=50

#store values of input and output
x=[] ; y=[]

for i in new_notes:
  for j in range(0,len(i)-timesteps):
    #input will be the current index + timestep
    #output will be the next index after timestep
    inp=i[j:j+timesteps] ; out=i[j+timesteps]

    #append the index value of respective notes
    x.append(list(map(lambda x:note2ind[x],inp)))
    y.append(note2ind[out])

x_new=np.array(x)
y_new=np.array(y)
print(x_new.shape)

(64673, 50)


In [ ]:
x_new = np.reshape(x_new,(len(x_new),timesteps,1))
y_new = np.reshape(y_new,(-1,1))

#splitting the input values into training and testing sets in 80:20 ratio
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_new,y_new,test_size=0.2, random_state =42 )

In [ ]:
#creating the model
model=Sequential()

#creating 2 stacked LSTM layer with dimension 256
model.add(LSTM(256,return_sequences=True,input_shape=(x_new.shape[1],x_new.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(256,activation='relu'))

#fully connected layer for the output with softmax activation
model.add(Dense(len(note2ind),activation='softmax'))
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 50, 256)             │         264,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 50, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 256)                 │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 76)                  │          19,532 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 874,828 (3.34 MB)

 Trainable params: 874,828 (3.34 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#creating the model
model_GRU=Sequential()

#creating 2 stacked LSTM layer with dimension 256
model_GRU.add(GRU(256,return_sequences=True,input_shape=(x_new.shape[1],x_new.shape[2])))
model_GRU.add(Dropout(0.2))
model_GRU.add(GRU(256))
model_GRU.add(Dropout(0.2))
model_GRU.add(Dense(256,activation='relu'))

#fully connected layer for the output with softmax activation
model_GRU.add(Dense(len(note2ind),activation='softmax'))
model_GRU.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                            │ (None, 50, 256)             │         198,912 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 50, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 76)                  │          19,532 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 678,988 (2.59 MB)

 Trainable params: 678,988 (2.59 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#compiling model with adam optimizer
model_GRU.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

#training model on training,validation and testing sets
model_GRU.fit(x_train,y_train,batch_size=128,epochs=80,validation_data=(x_test,y_test))

#saving model
model_GRU.save("MOD")



Epoch 1/80
405/405 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accuracy: 0.6588 - loss: 1.7965 - val_accuracy: 0.6574 - val_loss: 1.4263
Epoch 2/80
405/405 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.6637 - loss: 1.3844 - val_accuracy: 0.6646 - val_loss: 1.3666
Epoch 3/80
405/405 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.6698 - loss: 1.3371 - val_accuracy: 0.6692 - val_loss: 1.3307
Epoch 4/80
405/405 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.6761 - loss: 1.2967 - val_accuracy: 0.6745 - val_loss: 1.3042
Epoch 5/80
405/405 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.6809 - loss: 1.2634 - val_accuracy: 0.6796 - val_loss: 1.2658
Epoch 6/80
405/405 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.6875 - loss: 1.2161 - val_accuracy: 0.6885 - val_loss: 1.2164
Epoch 7/80
405/405 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.6978 - loss: 1.1643 - val_accuracy: 0.6949 - val_loss: 1.1830
Epoch 8/80
405/405 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.7113 - loss: 1.0971 - val

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=MOD_GRU.

In [ ]:
#loading model from saved models
model=load_model("MOD")

#generating random index
index = np.random.randint(0,len(x_test)-1)

music_pattern=x_test[index]

# making empty list for predicted notes
out_pred=[]

#iterating till 200 notes is generated
for i in range(200):
  #reshaping the music pattern
  music_pattern=music_pattern.reshape(1,len(music_pattern),1)

  #getting the note which has maximum probability of occurance
  pred_index = np.argmax(model.predict(music_pattern))
  out_pred.append(ind2note[pred_index])
  music_pattern = np.append(music_pattern,pred_index)

  #updating the music pattern with one timestamp ahead
  music_pattern = music_pattern[1:]

In [ ]:
#saving the predicted notes in output_notes
output_notes = []
for offset,pattern in enumerate(out_pred):
  #if pattern is a chord instance
  if ('.' in pattern) or pattern.isdigit():
    #split notes from the chord
    notes_in_chord = pattern.split('.')
    notes = []
    for current_note in notes_in_chord:
        i_curr_note=int(current_note)
        #cast the current note to Note object and
        #append the current note
        new_note = note.Note(i_curr_note)
        new_note.storedInstrument = instrument.Piano()
        notes.append(new_note)

    #cast the current note to Chord object
    #offset will be 1 step ahead from the previous note
    #as it will prevent notes to stack up
    new_chord = chord.Chord(notes)
    new_chord.offset = offset
    output_notes.append(new_chord)

  else:
    #cast the pattern to Note object apply the offset and
    #append the note
    new_note = note.Note(pattern)
    new_note.offset = offset
    new_note.storedInstrument = instrument.Piano()
    output_notes.append(new_note)

#save the midi file
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='AI_composed_music.mid')



'AI_composed_music.mid'